In [ ]:
import re
import pandas as pd

In [269]:
# tags from corpus
tagged_corpus = pd.read_csv("corpus_tags.tsv", sep="\t", names=["pmid", "start", "end", "word", "type", "umls_id"])
tagged_corpus = tagged_corpus.dropna()
tagged_corpus[0:91]

,pmid,start,end,word,type,umls_id
0,25763772,0,5,DCTN4,"T116,T123",C4308010
1,25763772,23,63,chronic Pseudomonas aeruginosa infection,T047,C0854135
2,25763772,67,82,cystic fibrosis,T047,C0010674
3,25763772,83,120,Pseudomonas aeruginosa (Pa) infection,T047,C0854135
4,25763772,124,139,cystic fibrosis,T047,C0010674
...,...,...,...,...,...,...
86,25763772,1607,1609,CF,T047,C0010674
87,25763772,1610,1618,patients,T101,C0030705
88,25763772,1627,1630,CPA,T047,C0854135
89,25763772,1643,1661,class II mutations,T045,C0026882


In [318]:
classes = []
with open("vocab_class.txt") as ftags:
    classes = [i.strip() for i in ftags.readlines()]
    ftags.close()
classes

['T047', 'T005', 'T007', 'T016']

In [398]:
def get_words_tags(text, pmid):
    start = 0 
    words = []
    tags = []  
    typ = "0"
    df = tagged_corpus[(tagged_corpus["pmid"]==int(pmid)) & (tagged_corpus["end"] <= start + len(text)) & (tagged_corpus["start"] >= start)]
    indices = df.index.values
    for word in text.split():
        words.append(word)
        for i in indices:
            text2 = df.iloc[i]["word"]
            if word in text2.split() and set(df.iloc[i]["type"].split(',')).intersection(classes):
                typ = df.iloc[i]["type"]
                break
            else:
                typ = "0"
        tags.append(typ)
    return words,tags
        
        
        
    

In [399]:
fr = open("para.txt")
fw = open("labelled_data.txt", "w")

title_regex = re.compile("^\d+\|t\|")
abstract_regex = re.compile("^\d+\|a\|") 

In [400]:
fr.seek(0,0)
for i in fr.readlines():
    line = i.strip()
    title_search = title_regex.search(line)
    abstract_search = abstract_regex.search(line)
    if title_search:
        title_line = title_regex.sub('',line)
    elif abstract_search:
        abstract = abstract_regex.sub('',line)
        pmid = int(abstract_search.group(0)[:-3])
        words,tags = get_words_tags(title_line,pmid)
        fw.write("\t".join(words)+"\n")
        fw.write("\t".join(tags)+"\n")
        title_size = len(title) + 1 
        for sentence in re.split("\.", abstract):
            if len(sentence) == 0:
                continue
            _size = abstract.index(sentence) + title_size
            words, tags = get_words_tags(sentence, pmid)
            fw.write("\t".join(words)+"\n")
            fw.write("\t".join(tags)+"\n")
        fout.write("\n")            
fw.close()
fr.close()